# Análise de Eficiência e Cálculo do Fator-X para Rodovias Federais Concedidas

In [14]:
################# Configurações preliminares
# import packages
from pystoned import CNLS
from pystoned import StoNED
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

# Elementos gráficos
from ipyfilechooser import FileChooser
import os
import plotly.graph_objects as go

# Necessário para utilizar os solvers do servidor NEOS
from pyomo import environ as pym
import os

## Configuração do otimizador
# provide an email address
os.environ["NEOS_EMAIL"] = "carlos.vneves@gmail.com"
solver_manager = pym.SolverManagerFactory("neos")

In [3]:
#from pyomo.environ import *
#model = ConcreteModel()
#model.x = Var(bounds=(1.0,10.0),initialize=5.0)
#model.y = Var(within=Binary)

#model.c1 = Constraint(expr=(model.x-4.0)**2 - model.x <= 50.0*(1-model.y))
#model.c2 = Constraint(expr=model.x*log(model.x)+5.0 <= 50.0*(model.y))

#model.objective = Objective(expr=model.x, sense=minimize)
#SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver='ipopt') 
#model.objective.display()


In [4]:
## Funções auxiliares
def SqB1Test(eps, n):
    """docstring"""
    Flag = 0
    # Calculate the first and second moments for each observation
    M2 = np.power(eps, 2)
    M3 = np.power(eps, 3)
    # calculate the average moment
    mM2 = np.mean(M2)
    mM3 = np.mean(M3)

    if 0 < mM3:
        mM3 = -0.0001
        Flag = 1

    teststat = mM3 / (mM2 ** (3 / 2))

    m = 1000

    S = np.zeros(n)
    teststatt = np.zeros(m)
    # Calculates the number of time a test statistic calculate from random
    # draws from a normal distribution is more negative the the one
    # calculated from the data set
    Flag1 = 0
    for j in range(m):
        for i in range(n):
            S[i] = np.random.normal(0, np.sqrt(mM2))

        M2t = np.power(S, 2)
        M3t = np.power(S, 3)
        mM2t = np.sum(M2t / (n - 1))
        mM3t = np.sum(M3t / (n - 1))
        teststatt[j] = mM3t / (mM2t) ** (3 / 2)
        if teststatt[j] < teststat:
            Flag1 = Flag1 + 1

    Pvalue = Flag1 / m

    return [teststat, teststatt, Pvalue, Flag]

def prod_func(model):
     # store the beta 
     beta = model.get_beta()
     # production df
     prod_df = pd.DataFrame(data=[beta[:,0],beta[:,1]])
     prod_df = prod_df.transpose()
     prod_df.columns = ["Custos","Receitas"]
     elasticity = prod_df["Custos"]/prod_df["Receitas"]
     prod_df.insert(2,"Elasticidade",elasticity)
     
     return prod_df

def calc_eff(model, model_name=''):
    # Resíduos
    eps = model.get_residual()
    ubenchmark = max(eps)
    yhat = model.get_frontier()
    # The purpose of this line is to impose monotonicity
    uCNLS = ubenchmark - eps
    # The purpose of this line is to impose monotonicity by
    phiC2NLS = yhat + ubenchmark
    # Farrel efficiency scores
    thetaC2NLS = yhat / phiC2NLS
    # Eficiência
    ustar = -(eps - ubenchmark)
    theta = np.exp(-ustar)

    return pd.DataFrame(theta, columns=[model_name+"_eff"])

# Base dados

- 21 contratos das três etapas de concessões de rodovias federais (2012-2016)

In [39]:
#load data from a dialog-box
# Create and display a FileChooser widget
#fc = FileChooser(os.getcwd())
#display(fc)
path = os.getcwd()  
data = pd.read_csv(path+'\\data\\data_eff.csv')

data["dmu"] = data["Unnamed: 0"]
data = data.drop("Unnamed: 0", axis=1)
data = data.set_index("dmu")
#data

fig = go.Figure(data=[go.Table( columnwidth = [800,400],
    header=dict(values=list(data.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[data.conc, data.avg, data.rec, data.cust, data.tar, 
                       data.ext, data.npp, data.etp, data.dEtp2, data.dEtp3],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(width=1000, height=600)
fig.update_traces(name="Base de dados.", visible=True,selector=dict(type='table'))
fig.show()

# Definição do Modelo:

- **Outputs:** avaliação geral do trecho concedido (AVG) de acordo com as pesquisas da CNT. No presente trabalho, utilizou-se a escala de um a cinco (nos moldes de uma escala *Likert*) para quantificar a avaliação dada para cada concessão. A nota de cada subtrecho dentro do trecho correspondente à rodovia concedida foi ponderada de acordo com a respectiva extensão, para então obtermos a nota geral da concessão.
- **Inputs:** receitas totais (RT), obtidas a partir das demonstrações financeiras auditadas das companhias, e disponibilizadas na página oficial da ANTT, divididos pelas respectivas extensões (EXT); custos totais (CT), obtidos a partir das demonstrações financeiras auditadas das companhias, e disponibilizadas na página oficial da ANTT, divididos pelas respectivas extensões (EXT).
- **Variáveis contextuais (Z):** o número de praças de pedágio por concessão (npp), a tarifa média a cada 100 km (tarm), e duas dummies dEtapa2 e dEtapa3 (verificar se o comportamento sofre influência da modelagem contratual de cada etapa).

\begin{align*}
& log(AVG) = f(log(RT),log(CT),log(Z)) \\
\end{align*}




In [7]:
# output
y = np.log(data['avg'])

# inputs
x1 = data['rec']
x1 = np.log(np.asmatrix(x1).T)
x2 = data['cust']
x2 = np.log(np.asmatrix(x2).T)
x = np.concatenate((x1, x2), axis=1)

# Modelo CNLS

No contexto VRS, a formulação (log-transformada) `CNLS` :

\begin{align*}
& \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
& \text{s.t.} \\
&  \text{ln}y_i = \text{ln}(\phi_i+1) + \varepsilon_i  \quad \forall i\\
& \phi_i  = \alpha_i+\beta_i^{'}X_i -1 \quad \forall i \\
&  \alpha_i + \beta_i^{'}X_i \le \alpha_j + \beta_j^{'}X_i  \quad  \forall i, j\\
&  \beta_i \ge 0 \quad  \forall i \\
\end{align*}

In [10]:
# define and solve the CNLS model
cnls_model = CNLS.CNLS(y, x, z=None, cet = "mult", fun = "prod", rts = "vrs")
cnls_model.optimize(remote=True)

Estimating the multiplicative model remotely with knitro solver


## Características da função de produção estimada

 - Produtividade marginal
 - Elasticidade de substituição
 
 O sumário das características da função de produção estimada constam da tabela a seguir.

In [58]:
prod_cnls = prod_func(cnls_model)

sum_cnls = round(prod_cnls.describe(),4)

fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","Custo marginal","Receita marginal","Elasticidade de substituição"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_cnls.index,sum_cnls.Custos, sum_cnls.Receitas, sum_cnls.Elasticidade],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

In [59]:
margcost_mean_cnls = round(prod_cnls.describe()["Custos"]["mean"],3)
margrev_mean_cnls = round(prod_cnls.describe()["Receitas"]["mean"],3)

Por exemplo, em média, 1% de incremento nos custos (que incluem os investimentos na *proxy* utilizada, aumenta a avaliação em {{margcost_mean_cnls}}%. Por outro lado, o aumento de 1% nas receitas, incrementa a avaliação em {{margrev_mean_cnls}}%.

## Calcula a eficiência para o Modelo CNLS

 - O sumário dos resultados da análise de eficiência pode ser visto na tabela a seguir.

In [66]:
eff_cnls = calc_eff(cnls_model, 'CNLS')
sum_eff_cnls =  round(eff_cnls.describe(),4)

In [68]:
fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","CNLS_eff"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_eff_cnls.index,sum_eff_cnls.CNLS_eff],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

## Teste do ajuste do modelo

- Os modelos paramétricos e semi-paramétricos adotam certas premissas sobre o processo gerador de dados. Portanto, após a sua estimação, é necessário verificar estas premissas não foram violadas.


In [69]:
eps = cnls_model.get_residual()
n = data.shape[0]

teststat, teststatt, Pvalue, Flag = SqB1Test(eps, n)


- O Teste de normalidade dos resíduos indentifica se a distribuição é assimétrica. Portanto, um p-valor < 0.05 indica que os resíduos são estatisticamente diferentes de uma distribuição normal no nível de 5% de significância, e que o modelo está inadequadamente especificado. 
- O resultado do teste para o modelo é: {{Pvalue}}

# Modelo StoNED 

In [73]:
st_model = StoNED.StoNED(y, x, z=None, cet = "mult", fun = "prod", rts = "vrs")
st_model.optimize(remote=True)

Estimating the multiplicative model remotely with knitro solver


## Fronteira de Produção

 - Produtividade marginal
 - Elasticidade de substituição
 
 O sumário das características da função de produção estimada constam da tabela a seguir.

In [74]:
prod_st = prod_func(st_model)
margcost_mean_st = round(prod_st.describe()["Custos"]["mean"],3)
margrev_mean_st = round(prod_st.describe()["Receitas"]["mean"],3)


In [75]:
sum_st = round(prod_st.describe(),4)

fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","Custo marginal","Receita marginal","Elasticidade de substituição"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_st.index,sum_st.Custos, sum_st.Receitas, sum_st.Elasticidade],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

## Calcula a eficiência para o modelo StoNED

In [76]:
eff_st = calc_eff(st_model, 'StoNED')
sum_eff_st = eff_st.describe()

In [77]:
fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","StoNED_eff"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_eff_cnls.index,sum_eff_cnls.CNLS_eff],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

## Testa o ajuste do modelo

In [78]:
eps = st_model.get_residual()
n = data.shape[0]

teststat, teststatt, Pvalue, Flag = SqB1Test(eps, n)



- O Teste de normalidade dos resíduos indentifica se a distribuição é assimétrica. Portanto, um p-valor < 0.05 indica que os resíduos são estatisticamente diferentes de uma distribuição normal no nível de 5% de significância, e que o modelo está inadequadamente especificado.
- O resultado do teste para o modelo é: {{Pvalue}}



# Modelo StoNEZD 

In [79]:
## Prepara os dados
# output
y = np.log(data["avg"])

# inputs
x1 = data["cust"]
x1 = np.log(np.asmatrix(x1).T)
x2 = data["rec"]
x2 = np.log(np.asmatrix(x2).T)
x = np.concatenate((x1, x2), axis=1)

# Z variables
# z = df['npp', 'ext', 'tar', 'dEtapa2', 'dEtapa3']
z1 = data["npp"]
z1 = np.asmatrix(z1).T

#z2 = data["ext"]
#z2 = np.asmatrix(z2).T

z3 = data["tar"]
z3 = np.asmatrix(z3).T

z4 = data["dEtp2"]
z4 = np.asmatrix(z4).T

z5 = data["dEtp3"]
z5 = np.asmatrix(z5).T

# z = np.concatenate((z1, z2, z3, z4, z5), axis=1)
z = np.concatenate((z1, z3, z4, z5), axis=1)

In [80]:
stz_model = StoNED.StoNED(y, x, z, cet = "mult", fun = "prod", rts = "vrs")
stz_model.optimize(remote=True)

Estimating the multiplicative model remotely with knitro solver


## Função de Produção

 - Produtividade marginal
 - Elasticidade de substituição
 
 O sumário das características da função de produção estimada constam da tabela a seguir.

In [85]:
prod_stz = prod_func(stz_model)
sum_stz = round(prod_stz.describe(),5)

In [86]:
fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","Custo marginal","Receita marginal","Elasticidade de substituição"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_stz.index,sum_stz.Custos, sum_stz.Receitas, sum_stz.Elasticidade],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

In [89]:
# retrive the technical inefficiency
eff_stz = np.asmatrix(stz_model.get_technical_inefficiency(method="MOM") ).T
eff_stz = pd.DataFrame(eff_stz, columns=["StoNEZD_eff"])
sum_eff_stz = round(eff_stz.describe(),4)

In [90]:
fig = go.Figure(data=[go.Table( 
    header=dict(values=list(["Estatísticas","StoNEZD_eff"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[sum_eff_stz.index,sum_eff_stz.StoNEZD_eff],
               fill_color='lavender',
               align='left'))
])
#fig.update_layout(width=1000, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

## Testa o ajuste do modelo 

In [91]:
eps = stz_model.get_residual()
n = data.shape[0]

teststat, teststatt, Pvalue, Flag = SqB1Test(eps, n)



- O Teste de normalidade dos resíduos indentifica se a distribuição é assimétrica. Portanto, um p-valor < 0.05 indica que os resíduos são estatisticamente diferentes de uma distribuição normal no nível de 5% de significância, e que o modelo está inadequadamente especificado.
- O resultado do teste para o modelo é: {{Pvalue}}

# Consolidação dos resultados da Análise de Eficiência

A nossa proposta para cálculo do indicador de eficiência para os contratos de concessão, baseia-se no cálculo dos escores de eficiência para cada concessionária através de diversos métodos, o cálculo da sua média aritmética (limitando os escores ao mínimo de 50\%) e a eliminação de \emph{outliers} (análise de correlação), para a obtenção de um indicador de eficiência final para cada contrato de concessão.

\begin{align*}
							& E_i = max\{\bar{E}_i;0.50\} \\
							& E_i = \{E \in R | E = 0.50,...,1.00\} \\
							& \bar{E}_i =  \frac{1}{n} \sum_{k=1}^{n} E_{k}^{i} \quad \text{s.t.} \, \rho_{E_{k,DEA}^i} > |0.50|\\
							& E_{k}^{i} \in \{E_{DEA}^{i,a};E_{SFA}^{i,b};E_{StoNEZD}^{i,c}\} \\
							& a \in \{DEA, DEA_{bc}, DEA_{sw}\} \\
							& b \in \{SFA_{ols},SFA_{cols}, SFA_{cmad}, SFA_{hn}, SFA_{hnx}, SFA_{exp}, SFA_{expx}\} \\
							& c \in \{CNLS, StoNED, StoNEZD\}
						\end{align*}

## Resultados consolidados

In [92]:
dea_res = pd.read_csv(path+'\\data\\dea_results.csv')
sfa_res = pd.read_csv(path+'\\data\\sfa_results.csv')

dea_res = dea_res.sort_values('conc', ascending=True)
dea_res["dmu"] = range(1,22) 
dea_res = dea_res.set_index("dmu") 

sfa_res = sfa_res.sort_values('conc', ascending=True)
sfa_res["dmu"] = range(1,22) 
sfa_res = sfa_res.set_index("dmu") 
#sfa_res



In [101]:
# junta os dados em um mesmo dataframe
df_eff = np.concatenate((dea_res.iloc[:,2:5],
 sfa_res.iloc[:,11:18],eff_cnls,eff_st,eff_stz), axis=1)
#renomeia as colunas
df_eff = pd.DataFrame(df_eff, columns=["eff_dea","eff_deabc","eff_deasw",
"eff_sfaols","eff_sfacols","eff_sfacmad","eff_sfahn","eff_sfaexp",
"eff_sfahnx","eff_sfaexpx","eff_cnls","eff_st","eff_stz"])
#df_eff = pd.concat([data,df_eff], axis=1)
# organiza os resultados por DMU
df_eff["dmu"] = range(1,22) 
#df_eff["dmu"] = data.index
df_eff = df_eff.set_index("dmu")
#df_eff = df_eff.drop(["Unnamed: 0"], axis=1)


In [117]:
data_eff_sum = round(df_eff.describe(),4)

fig = go.Figure(data=[go.Table(  columnwidth = [500,400],
    header=dict(values=list(["Estatísticas","eff_dea","eff_deabc","eff_deasw",
"eff_sfaols","eff_sfacols","eff_sfacmad","eff_sfahn","eff_sfaexp",
"eff_sfahnx","eff_sfaexpx","eff_cnls","eff_st","eff_stz"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[data_eff_sum.index,
                       data_eff_sum.eff_dea,data_eff_sum.eff_deabc,data_eff_sum.eff_deabc,
                      data_eff_sum.eff_deasw,data_eff_sum.eff_sfaols,data_eff_sum.eff_sfacmad,
                      data_eff_sum.eff_sfahn,data_eff_sum.eff_sfaexp,data_eff_sum.eff_sfahnx,
                      data_eff_sum.eff_sfaexpx,data_eff_sum.eff_cnls,data_eff_sum.eff_st,
                       data_eff_sum.eff_stz],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(width=1280, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

## Análise de Correlação para eliminação de outliers

In [121]:
df_eff_corr = round(df_eff.corr(),4)

fig = go.Figure(data=[go.Table(columnwidth = [500,400],
    header=dict(values=list(["","eff_dea","eff_deabc","eff_deasw",
"eff_sfaols","eff_sfacols","eff_sfacmad","eff_sfahn","eff_sfaexp",
"eff_sfahnx","eff_sfaexpx","eff_cnls","eff_st","eff_stz"]),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_eff_corr.index,
                       df_eff_corr.eff_dea,df_eff_corr.eff_deabc,df_eff_corr.eff_deabc,
                      df_eff_corr.eff_deasw,df_eff_corr.eff_sfaols,df_eff_corr.eff_sfacmad,
                      df_eff_corr.eff_sfahn,df_eff_corr.eff_sfaexp,df_eff_corr.eff_sfahnx,
                      df_eff_corr.eff_sfaexpx,df_eff_corr.eff_cnls,df_eff_corr.eff_st,
                      df_eff_corr.eff_stz],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(width=1280, height=600)
fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
fig.show()

In [133]:
eff_corr = pd.DataFrame(np.abs(df_eff.corr()['eff_dea']) > np.abs(0.5))
eff_corr = eff_corr[eff_corr['eff_dea']==True]
eff_corr = eff_corr.reset_index()
df_eff_corr = df_eff[eff_corr["index"]]
df_eff_corr

# fig = go.Figure(data=[go.Table(columnwidth = [500,400],
#     header=dict(values=list(["dmu","eff_dea","eff_deabc","eff_deasw",
# "eff_sfaols","eff_sfacols","eff_sfacmad","eff_sfahn","eff_sfaexp",
# "eff_sfahnx","eff_sfaexpx","eff_cnls","eff_st","eff_stz"]),
#                 fill_color='paleturquoise',
#                 align='left'),
#     cells=dict(values=[{"name": i, "id": i} for i in sorted(df_eff_corr.columns)],
#                fill_color='lavender',
#                align='left'))
# ])
# fig.update_layout(width=1280, height=600)
# fig.update_traces(name="Sumário dos resultados.", visible=True,selector=dict(type='table'))
# fig.show()
# df_eff_corr[list(df_eff_corr.columns)]

,eff_dea,eff_deabc,eff_deasw,eff_sfacmad,eff_sfahn,eff_sfaexp,eff_sfahnx,eff_sfaexpx,eff_cnls,eff_st,eff_stz
dmu,,,,,,,,,,,
1,0.857499,0.856277,0.850629,0.787510,0.886725,0.886725,0.886725,0.886725,0.820308,0.820308,0.867607
2,0.791056,0.790328,0.785609,0.750712,0.829357,0.829357,0.829357,0.829357,0.899329,0.899329,0.918683
3,0.902053,0.901174,0.895812,0.877043,0.805576,0.805576,0.805576,0.805576,0.891024,0.891024,0.907751
4,0.828529,0.817297,0.813307,0.840562,0.833721,0.833721,0.833721,0.833721,0.859879,0.859879,0.876706
5,0.998202,0.997679,0.992109,0.981404,0.718176,0.718176,0.718176,0.718176,0.968647,0.968647,0.986645
6,0.998202,0.997834,0.994001,0.877043,0.795164,0.795164,0.795164,0.795164,0.913621,0.913621,0.930712
7,0.851806,0.811237,0.801392,0.843986,0.826262,0.826262,0.826262,0.826262,0.911447,0.911447,0.928456
8,0.791056,0.790755,0.787159,0.698836,1.000000,1.000000,1.000000,1.000000,0.751630,0.751630,0.788509
9,0.846472,0.846140,0.841994,0.869712,0.807649,0.807649,0.807649,0.807649,0.893077,0.893077,0.927436


# Cálculo do Fator-X

## Parâmetros Regulatórios



\begin{align*}
        
            & X_t^{i} = P_i(t) \delta_i, t =1,2,...,5 \\
            & \delta_i^{sup} = 0.2; \\
            & p_{inf} = 0.10 \;;\; p_{sup} = 0.35; \\
            & \textbf{Regra de decaimento exponencial:} 	\\
            & P_i(t) = 0.4787 e^{-0.3132 t} \\
            & \textbf{Valores discretos:} \\
            & P_i(t) = \{0.35, 0.22, 0.19, 0.14, 0.1\} \\
            &\sum_{i=1}^{5} P_i(t) = 1  \\
        
\end{align*}

In [134]:
delta_sup = 0.2
p_inf = 0.10
p_sup = 0.35
T = 5

def exp_decay(p_inf, p_sup, T):
    b = np.log(p_sup/p_inf)*1/(1-T)
    a = p_inf/(np.exp(b*T))
    t = range(1,T+1)
    y = a*np.exp(b*t)
    y[0] = p_sup
    y[T-1] = p_inf
    y = np.round(y,2)    
    sum_P = np.sum(y)

    
    if(sum_P > 1):
        diff = sum_P - 1
        #d = diff/(T-1)
        y[1] = y[1] - diff

    
    return [a,b,y]


    



In [198]:
a,b,P = exp_decay(p_inf, p_sup, T)


In [203]:
#plt.plot(range(1,6),P,'r-')
import plotly.express as px
t = range(1,6)
func =  a*np.exp(b*t)

# Use directly Columns as argument. You can use tab completion for this!
fig = px.line(x=t,y=[P,func],labels={"t": "Tempo", "P": "P_t", "func":"função exponencial"})

fig.show()




In [ ]:
#np.sum(P)

In [ ]:
#E_i = 0.85
#delta_i = min(1-E_i,delta_sup)
#delta_i

In [ ]:
#X_t = np.multiply(P,delta_i)
#X_t

In [ ]:
#sum(X_t)

In [204]:
def fator_X(df_eff,P, delta_sup): 
    E_bar = df_eff.mean(axis=1)
    n = len(E_bar)
    T = len(P)
    delta_i = np.zeros(n)
    for i in range(n):
        delta_i[i] = min(1-E_bar[i+1],delta_sup)
    
    Pm = np.repeat(np.asmatrix(P), n, axis=0)

    X_t = np.multiply(np.asmatrix(delta_i).T,Pm)
    cols = ["t="+ str(x) for x in range(1,T+1)]
    X_t = pd.DataFrame(X_t,columns=cols)
    X_t["dmu"] = range(1,n+1)      
    X_t = X_t.set_index("dmu")
    
    return X_t


In [214]:
X_t = fator_X(df_eff_corr,P, delta_sup)
X_t.describe()

,t=1,t=2,t=3,t=4,t=5
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.046105,0.028980,0.025028,0.018442,0.013173
std,0.007015,0.004409,0.003808,0.002806,0.002004
min,0.035748,0.022470,0.019406,0.014299,0.010214
25%,0.040652,0.025553,0.022068,0.016261,0.011615
50%,0.043746,0.027498,0.023748,0.017499,0.012499
75%,0.052028,0.032704,0.028244,0.020811,0.014865
max,0.059235,0.037234,0.032156,0.023694,0.016924


In [216]:
fig = px.histogram(X_t)
fig.show()